In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape
input_shape = (480, 640, 3)

# Define the input tensor
inputs = layers.Input(shape=input_shape)

# Define the CNN architecture
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model = models.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='AUC')



In [2]:
import numpy as np
X = np.load('8x640_480.npy')
Y = np.load('8y640_480.npy')
def swap_zeros_ones(arr):
    arr[arr == 0] = 2
    arr[arr == 1] = 0
    arr[arr == 2] = 1
    return arr
Y = swap_zeros_ones(Y)

In [3]:
y = []
for i in Y:
    if np.average(i) == 0:
        y.append(0)
    else:
        y.append(1)

In [4]:
y = np.array(y)

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    X, y, test_size=0.2
)

# Define the training data generator with data augmentation
train_data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Define the validation data generator without data augmentation or rescaling
val_data_gen = ImageDataGenerator()

# Create the training data iterator
train_data_iterator = train_data_gen.flow(
    train_images, train_labels,
    batch_size=32
)

# Create the validation data iterator
val_data_iterator = val_data_gen.flow(
    val_images, val_labels,
    batch_size=32
)

# # Train the model using the training and validation data iterators
# model.fit(
#     train_data_iterator,
#     epochs=epochs,
#     steps_per_epoch=len(train_images) // batch_size,
#     validation_data=val_data_iterator,
#     validation_steps=len(val_images) // batch_size
# )


In [6]:
from keras.callbacks import EarlyStopping
patience = 20
early_stopping = EarlyStopping(monitor='val_auc', restore_best_weights = True, min_delta=0.0001, patience= patience, verbose = 1, mode = 'max')

In [7]:
# Train the model using the training and validation data iterators
history = model.fit(
    train_data_iterator,
    epochs=500,
    batch_size=32,
    validation_data=val_data_iterator,
    callbacks=[early_stopping]
)

Epoch 1/500
104/104 [==============================] - 139s 1s/step - loss: 22.4329 - auc: 0.7213 - val_loss: 0.5166 - val_auc: 0.8539
Epoch 2/500
104/104 [==============================] - 138s 1s/step - loss: 0.4810 - auc: 0.8527 - val_loss: 0.4313 - val_auc: 0.8880
Epoch 3/500
104/104 [==============================] - 138s 1s/step - loss: 0.5572 - auc: 0.8350 - val_loss: 0.5263 - val_auc: 0.8149
Epoch 4/500
104/104 [==============================] - 138s 1s/step - loss: 0.5240 - auc: 0.8162 - val_loss: 0.5277 - val_auc: 0.8698
Epoch 5/500
104/104 [==============================] - 138s 1s/step - loss: 0.4652 - auc: 0.8643 - val_loss: 0.4807 - val_auc: 0.8531
Epoch 6/500
104/104 [==============================] - 139s 1s/step - loss: 0.4420 - auc: 0.8719 - val_loss: 0.3854 - val_auc: 0.9031
Epoch 7/500
104/104 [==============================] - 138s 1s/step - loss: 0.4088 - auc: 0.8907 - val_loss: 0.3254 - val_auc: 0.9277
Epoch 8/500
104/104 [==============================] - 138s 1

Epoch 62/500
104/104 [==============================] - 139s 1s/step - loss: 0.2241 - auc: 0.9656 - val_loss: 0.1996 - val_auc: 0.9785
Epoch 63/500
104/104 [==============================] - 139s 1s/step - loss: 0.2223 - auc: 0.9677 - val_loss: 0.2318 - val_auc: 0.9634
Epoch 64/500
104/104 [==============================] - 137s 1s/step - loss: 0.2732 - auc: 0.9549 - val_loss: 0.2517 - val_auc: 0.9703
Epoch 65/500
104/104 [==============================] - 137s 1s/step - loss: 0.2313 - auc: 0.9668 - val_loss: 0.1887 - val_auc: 0.9801
Epoch 66/500
104/104 [==============================] - 139s 1s/step - loss: 0.2264 - auc: 0.9665 - val_loss: 0.1688 - val_auc: 0.9856
Epoch 67/500
104/104 [==============================] - 137s 1s/step - loss: 0.2527 - auc: 0.9620 - val_loss: 0.2561 - val_auc: 0.9642
Epoch 68/500
104/104 [==============================] - 137s 1s/step - loss: 0.2587 - auc: 0.9597 - val_loss: 0.2439 - val_auc: 0.9749
Epoch 69/500
104/104 [==============================] -

In [16]:
model.save('binary_classificator_H_v5_640x480')

INFO:tensorflow:Assets written to: binary_classificator_H_v5_640x480\assets


INFO:tensorflow:Assets written to: binary_classificator_H_v5_640x480\assets
